# Prepare Data for EDA and Models

Read and combine data from soures. Group data for EDA and Modeling

In [1]:
# Imports
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

In [2]:
def combine_files(path_to_files):
    '''
    Combine several .csv files into one Data Frame
    Input: string, the path to the folder with files
    Return: Data Frame with all files from folder concatenated, excluding created in this notebook
    '''
    df = pd.DataFrame()
    file_list = listdir('../data/')
    for file in file_list:
        if ('.csv' in file) & (file != 'all_states.csv')  & (file != 'most_restricted.csv') & (file != 'combined_states.csv') & (file != 'least_restricted.csv'):
            df = pd.concat([df, pd.read_csv(f'../data/{file}')])
    return df

---

In [3]:
all_states = combine_files('../data/')
all_states.head()

,state,week,depression,anxiety,addiction,counselling,mental_health,stay_at_home,mask_mandate,gatherings_banned,business_closures,travel_restrictions
0,Alaska,2017-12-31,28,46,24,37,0,0,0,0,0,0
1,Alaska,2018-01-07,37,45,25,65,37,0,0,0,0,0
2,Alaska,2018-01-14,10,46,16,53,30,0,0,0,0,0
3,Alaska,2018-01-21,33,41,10,35,14,0,0,0,0,0
4,Alaska,2018-01-28,32,24,0,40,0,0,0,0,0,0


In [4]:
# updating the week column to be datetime type
all_states['week'] = pd.to_datetime(all_states['week'], format = '%Y-%m-%d')
all_states.dtypes

state                          object
week                   datetime64[ns]
depression                      int64
anxiety                         int64
addiction                       int64
counselling                     int64
mental_health                   int64
stay_at_home                    int64
mask_mandate                    int64
gatherings_banned               int64
business_closures               int64
travel_restrictions             int64
dtype: object

In [13]:
# Spliting the data in states with most and least restrictions

all_states['covid_restrictions'] = all_states['state'].apply(lambda x: 'Most Restrictions' if (x == 'Washington') | (x == 'Massachusetts') | (x == 'New York') | (x == 'California') | (x == 'Hawaii') | (x == 'Alaska') else 'Least Restrictions')

In [14]:
# Save combined data to csv
all_states.to_csv('../data/all_states.csv', index=False)

---

In [15]:
# Get most restricted states to separate dataframe
most_restricted = all_states[all_states['covid_restrictions'] == 'Most Restrictions'].groupby('week').mean()
most_restricted.sort_index(inplace=True)
most_restricted.head()

,depression,anxiety,addiction,counselling,mental_health,stay_at_home,mask_mandate,gatherings_banned,business_closures,travel_restrictions
week,,,,,,,,,,
2017-12-31,54.333333,66.000000,26.500000,31.166667,24.000000,0.0,0.0,0.0,0.0,0.0
2018-01-07,59.333333,66.666667,26.000000,39.500000,33.833333,0.0,0.0,0.0,0.0,0.0
2018-01-14,54.666667,62.333333,23.500000,39.166667,34.500000,0.0,0.0,0.0,0.0,0.0
2018-01-21,64.333333,66.500000,22.333333,38.500000,34.333333,0.0,0.0,0.0,0.0,0.0
2018-01-28,67.000000,65.000000,24.833333,36.166667,31.833333,0.0,0.0,0.0,0.0,0.0


In [16]:
# Save Most Restricted to csv
most_restricted.to_csv('../data/most_restricted.csv')

In [17]:
# Get least restricted states to separate dataframe
least_restricted = all_states[all_states['covid_restrictions'] == 'Least Restrictions'].groupby('week').mean()
least_restricted.sort_index(inplace=True)
least_restricted.head()

,depression,anxiety,addiction,counselling,mental_health,stay_at_home,mask_mandate,gatherings_banned,business_closures,travel_restrictions
week,,,,,,,,,,
2017-12-31,44.50,53.25,19.75,28.25,10.75,0.0,0.0,0.0,0.0,0.0
2018-01-07,53.75,55.75,19.25,22.25,16.75,0.0,0.0,0.0,0.0,0.0
2018-01-14,52.25,54.25,21.00,32.75,22.25,0.0,0.0,0.0,0.0,0.0
2018-01-21,62.50,58.75,24.50,32.75,19.00,0.0,0.0,0.0,0.0,0.0
2018-01-28,52.75,58.25,21.75,35.75,16.25,0.0,0.0,0.0,0.0,0.0


In [18]:
# Save Least Restricted to csv
least_restricted.to_csv('../data/least_restricted.csv')

---

In [19]:
# Combine all together to model COVID-19 impact overall
combined_states = all_states.groupby('week').mean()
combined_states.sort_index(inplace=True)
combined_states.head()

,depression,anxiety,addiction,counselling,mental_health,stay_at_home,mask_mandate,gatherings_banned,business_closures,travel_restrictions
week,,,,,,,,,,
2017-12-31,50.4,60.9,23.8,30.0,18.7,0.0,0.0,0.0,0.0,0.0
2018-01-07,57.1,62.3,23.3,32.6,27.0,0.0,0.0,0.0,0.0,0.0
2018-01-14,53.7,59.1,22.5,36.6,29.6,0.0,0.0,0.0,0.0,0.0
2018-01-21,63.6,63.4,23.2,36.2,28.2,0.0,0.0,0.0,0.0,0.0
2018-01-28,61.3,62.3,23.6,36.0,25.6,0.0,0.0,0.0,0.0,0.0


In [20]:
# Save All States searches combined
combined_states.to_csv('../data/combined_states.csv')

We prepared several different files for future EDA and Modeling. In the next notebook we will look at the data.